In [62]:
# Import necessary libraries
import dspy
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Initialize the OpenAI language model
cheap_lm = dspy.LM("openai/gpt-5-mini", api_key=os.getenv("OPENAI_API_KEY"), temperature=1, max_tokens=16000, reasoning_effort="low")
smart_lm = dspy.LM("openai/gpt-5", api_key=os.getenv("OPENAI_API_KEY"), temperature=1, max_tokens=16000, reasoning_effort="high")

# Define the Comedian class signature
class ComedianSignature(dspy.Signature):
    """
    Tell me a funny joke.
    """
    topic: str = dspy.InputField(description="The message to analyze")
    joke: str = dspy.OutputField(description="A funny joke about the topic")

# Define the comedian program
comedian = dspy.Predict(ComedianSignature)

# Create instances of Comedian with different LMs
with dspy.context(lm=cheap_lm):
    cheap_response = comedian(topic="programming")

with dspy.context(lm=smart_lm):
    smart_response = comedian(topic="programming")

print("Cheap LM response:")
print(cheap_response.joke)

print("\nSmart LM response:") 
print(smart_response.joke)

Cheap LM response:
Why do programmers prefer dark mode? Because light attracts bugs — and they already have enough features labeled "undocumented."

Smart LM response:
There are only two hard things in programming: cache invalidation, naming things, and off-by-one errors.


In [63]:
from typing import Literal, Any

dspy.configure(lm=cheap_lm)

# Define the Persona class signature
class AudienceSignature(dspy.Signature):
    """
    Decide if the joke is funny.
    """
    joke: str = dspy.InputField(description="A joke to evaluate")
    profiles: list[str] = dspy.InputField(description="Profiles of the audience members")
    reactions: list[str] = dspy.OutputField(description="Short reaction from each audience member explaining their inner thought process when hearing the joke (one per profile, same order)")
    responses: list[Literal["hilarious", "funny", "meh", "not funny", "offensive"]] = dspy.OutputField(description="Rating from each audience member (one per profile, same order)")

# Create audience evaluator class
class AudienceEvaluator(dspy.Module):
    """
    DSPy module that defines five audience personas and queries them
    to produce an aggregate funniness score for a given joke.
    """
    def __init__(self):
        super().__init__()
        # Define audience personas inside the module (single source of truth)
        self.profiles = [
            "35-year-old comedy club owner who has seen every major standup special and demands originality",
            "42-year-old comedy critic who writes for The New Yorker and analyzes joke structure and social commentary",
            "38-year-old professional comedian who performs nightly and is tired of hacky material",
            "45-year-old comedy festival curator who looks for unique voices and fresh perspectives",
            "40-year-old comedy writing professor who teaches advanced joke construction and timing"
        ]
        # Predictor that asks personas to rate the joke using chain-of-thought
        self.audience = dspy.Predict(AudienceSignature)
        # Map qualitative ratings to a numeric score
        self.rating_scores = {
            "hilarious": 5,
            "funny": 4,
            "meh": 3, 
            "not funny": 2,
            "offensive": 1
        }

    def forward(self, joke: str) -> dict[str, Any]:
        """
        Query each persona and return a structured result including average score,
        per-persona ratings and reactions, and the profiles used.
        """
        # Get ratings from all personas at once
        response = self.audience(joke=joke, profiles=self.profiles)
        
        # Calculate total score from list of ratings
        total_score = sum(self.rating_scores[rating] for rating in response.responses)
            
        # Calculate average score out of 5 to 2 decimal places
        avg_score = round(total_score / len(self.profiles), 2)
        
        # Return structured result for downstream debugging/analysis
        return {
            "avg_score": avg_score,
            "responses": list(response.responses),
            "reactions": list(response.reactions),
            "profiles": list(self.profiles),
            "joke": joke,
        }

# Create evaluator instance        
audience = AudienceEvaluator()

# Example usage
joke = cheap_response.joke
print("Joke: ", joke)
result = audience(joke)
print(f"Average audience score for joke: {result['avg_score']}/5")

print("--------------------------------")

# Access the full structured object for debugging/analysis
audience_result_1 = result

# Let's try another example
joke2 = smart_response.joke

print("Joke2: ", joke2)

result2 = audience(joke2)
print(f"Average audience score for joke2: {result2['avg_score']}/5")

# Access the full structured object for the second joke
audience_result_2 = result2



Joke:  Why do programmers prefer dark mode? Because light attracts bugs — and they already have enough features labeled "undocumented."
Average audience score for joke: 2.8/5
--------------------------------
Joke2:  There are only two hard things in programming: cache invalidation, naming things, and off-by-one errors.
Average audience score for joke2: 3.2/5


In [64]:
def metric_with_feedback(example, pred, trace=None, pred_name=None, pred_trace=None):
    """
    Use the precomputed audience score from AudienceEvaluator and set feedback
    to a concatenated string of "<profile>: <reaction>" for each persona (in order).
    """
    audience = AudienceEvaluator()
    result = audience(pred.joke)
    avg_score = result["avg_score"] / 5
    reactions = result["reactions"]
    profiles = result["profiles"]
    feedback = " ".join(f"Persona: {p}\nReaction: {r}\n\n---\n" for p, r in zip(profiles, reactions))

    if pred_name is None:
        return avg_score
    else:
        # give feedback to the model optimizing the score
        return dspy.Prediction(score=avg_score, feedback=feedback)


example = dspy.Example(
    topic="programming",
    joke="Why does a programmer prefer dark mode? Because light attracts bugs!",
)


metric_with_feedback(example, example, trace='feedback')


0.6

In [65]:
# Dataset of professional comedian jokes (labeled as funny)
# Source: Various famous comedians
# https://inews.co.uk/light-relief/jokes/ricky-gervais-jokes-best-golden-globes-2020-host-controversial-funniest-the-office-135797
# https://www.blackpoolgrand.co.uk/funniest-jokes-one-liners/
# https://www.vulture.com/2018/01/dave-chappelle-bird-revelation-equanimity-best-jokes.html
# https://www.scotsman.com/heritage-and-retro/heritage/billy-connollys-best-jokes-80-of-the-big-yins-funniest-jokes-and-one-liners-4458332
# https://inews.co.uk/light-relief/jokes/funny-jokes-110-funniest-best-one-liners-192413

funny_jokes = [
    {"topic": "Fishing", "joke": "Give a man a fish, and he'll probably follow you home expecting more fish.", "comedian": "Ricky Gervais"},
    {"topic": "Family", "joke": "Where there's a will – there's a relative!", "comedian": "Ricky Gervais"},
    {"topic": "Holidays", "joke": "1st of December, World Aids Day….I don't think it'll ever take off like Christmas.", "comedian": "Ricky Gervais"},
    {"topic": "Drinking", "joke": "I like a drink as much as the next man. Unless the next man is Mel Gibson.", "comedian": "Ricky Gervais"},
    {"topic": "Celebrity", "joke": "It's gonna be a night of partying and heavy drinking. Or as Charlie calls it: breakfast.", "comedian": "Ricky Gervais"},
    {"topic": "Movies", "joke": "It seems like everything this year was three-dimensional, except the characters in The Tourist.", "comedian": "Ricky Gervais"},
    {"topic": "Religion", "joke": "You won't burn in hell. But be nice anyway.", "comedian": "Ricky Gervais"},
    {"topic": "Inspiration", "joke": "My greatest hero is Nelson Mandela. What a man. Incarcerated for 25 years, he was released in 1990 and he hasn't reoffended. I think he's going straight, which shows you prison does work.", "comedian": "Ricky Gervais"},
    {"topic": "Philosophy", "joke": "Remember, when you are dead, you do not know you are dead. It is only painful for others. The same applies when you are stupid.", "comedian": "Ricky Gervais"},
    {"topic": "Life", "joke": "Mondays are fine. It's your life that sucks.", "comedian": "Ricky Gervais"},
    {"topic": "Religion", "joke": "Remember, if you don't sin, then Jesus died for nothing.", "comedian": "Ricky Gervais"},
    {"topic": "Activism", "joke": "I could solve the world's problems if I… cared.", "comedian": "Ricky Gervais"},
    {"topic": "Identity", "joke": "I can have a go at the French cause I'm half French half English with a stupid name like Gervais. No I am, I'm half French half English and um I've got qualities of both, French and English which is good, so um… I am crap in bed but at least I've got bad breath.", "comedian": "Ricky Gervais"},
    {"topic": "Military", "joke": "Do commandos not wear pants? They must wear pants, don't they?", "comedian": "Ricky Gervais"},
    {"topic": "Equality", "joke": "Same sex marriage is not a gay privilege, it's equal rights. Privilege would be something like gay people not paying taxes. Like churches don't.", "comedian": "Ricky Gervais"},
    {"topic": "Folklore", "joke": "I've never worked out what the moral of Humpty Dumpty is. I can only think of: Don't sit on a wall, if you're an egg.", "comedian": "Ricky Gervais"},
    {"topic": "Employment", "joke": "Avoid employing unlucky people – throw half of the pile of CVs in the bin without reading them.", "comedian": "Ricky Gervais"},
    {"topic": "Awards", "joke": "For any of you who don't know, the Golden Globes are just like the Oscars, but without all that esteem. The Golden Globes are to the Oscars what Kim Kardashian is to Kate Middleton. A bit louder, a bit trashier, a bit drunker, and more easily bought.", "comedian": "Ricky Gervais"},
    {"topic": "Workplace", "joke": "If your boss is getting you down, look at him through the prongs of a fork and imagine him in jail.", "comedian": "Ricky Gervais"},
    {"topic": "Humor", "joke": "I can't find someone funny whom I don't like. Hitler told great jokes.", "comedian": "Ricky Gervais"},
    {"topic": "Culture", "joke": "America champions the underdog. We champion the under dog until he's not the underdog anymore, and he annoys us.", "comedian": "Ricky Gervais"},
    {"topic": "Betrayal", "joke": "You have to be 100% behind someone, before you can stab them in the back.", "comedian": "Ricky Gervais"},
    {"topic": "Health", "joke": "Remember, being healthy is basically dying as slowly as possible.", "comedian": "Ricky Gervais"},
    {"topic": "Atheism", "joke": "I'd like to thank God for making me an atheist.", "comedian": "Ricky Gervais"},
    {"topic": "Music Industry", "joke": "Piracy doesn't kill music, boy bands do.", "comedian": "Ricky Gervais"},
    {"topic": "Wealth", "joke": "My wealth and happiness would suggest that God definitely does love me. If he existed of course. Which he doesn't.", "comedian": "Ricky Gervais"},
    {"topic": "Social Media", "joke": "Following someone on Twitter and asking them to tweet about something else is like stalking someone and asking them to go a different route.", "comedian": "Ricky Gervais"},
    {"topic": "Fame", "joke": "Please don't worship me. I'm just an ordinary guy, with lots of followers trying to spread my message. Sort of like Jesus Christ I guess.", "comedian": "Ricky Gervais"},
    {"topic": "Technology", "joke": "iPhones are Barbie Dolls for grown men. You carry them round, dress them up in little outfits, accessorise, & get a new one every year.", "comedian": "Ricky Gervais"},
    {"topic": "Generosity", "joke": "Give a man a fish, and he'll probably follow you home expecting more fish.", "comedian": "Ricky Gervais"},
    {"topic": "Environment", "joke": "It seems to be true, particularly in middle America, that those most militant about using up fossil fuels, don't actually believe in fossils", "comedian": "Ricky Gervais"},
    {"topic": "Drinking", "joke": "My father drank so heavily, when he blew on the birthday cake he lit the candles.", "comedian": "Les Dawson"},
    {"topic": "Police", "joke": "I was in my car driving back from work. A police officer pulled me over and knocked on my window. I said, 'One minute I'm on the phone.'", "comedian": "Alan Carr"},
    {"topic": "Overthinking", "joke": "I worry about ridiculous things, you know, how does a guy who drives a snowplough get to work in the morning… that can keep me awake for days.", "comedian": "Billy Connolly"},
    {"topic": "Relationships", "joke": "I used to go out with a giraffe. Used to take it to the pictures and that. You'd always get some bloke complaining that he couldn't see the screen.", "comedian": "Paul Merton"},
    {"topic": "Music", "joke": "Here's a picture of me with REM. That's me in the corner.", "comedian": "Milton Jones"},
    {"topic": "Optimism", "joke": "People say 'Bill, are you an optimist?' And I say, 'I hope so.'", "comedian": "Bill Bailey"},
    {"topic": "Customer Service", "joke": "I rang up British Telecom and said: 'I want to report a nuisance caller.' He said: 'Not you again.'", "comedian": "Tim Vine"},
    {"topic": "Obesity", "joke": "Life is like a box of chocolates. It doesn't last long if you're fat.", "comedian": "Joe Lycett"},
    {"topic": "Religion", "joke": "We weren't very religious. On Hanukkah, my mother had our menorah on a dimmer.", "comedian": "Richard Lewis"},
    {"topic": "Beauty", "joke": "My girlfriend is absolutely beautiful. Body like a Greek statue – completely pale, no arms.", "comedian": "Phil Wang"},
    {"topic": "Weather", "joke": "Normally you have news, weather and travel. But not on snow day. On a snow day, the news is weather is travel.", "comedian": "Michael McIntyre"},
    {"topic": "Personal Improvement", "joke": "I bought myself some glasses. My observational comedy improved.", "comedian": "Sara Pascoe"},
    {"topic": "Sports", "joke": "If I was an Olympic athlete, I'd rather come in last than win the silver medal. You win the gold, you feel good. You win the bronze, you think, 'at least I got something.' But you win that silver, that's like, 'Congratulations, you almost won! Of all the losers, you came in first! You're the number one loser! No one lost ahead of you!'", "comedian": "Jerry Seinfeld"},
    {"topic": "Identity", "joke": "My star sign is Pyrex. I was a test-tube baby.", "comedian": "Billy Connolly"},
    {"topic": "Marriage", "joke": "I always take my wife morning tea in my pyjamas. But is she grateful? No, she says she'd rather have it in a cup.", "comedian": "Eric Morecambe"},
    {"topic": "Shopping", "joke": "A man walks into a chemist's and says, 'Can I have a bar of soap, please?' The chemist says, 'Do you want it scented?' And the man says, 'No, I'll take it with me now.'", "comedian": "Ronnie Barker"},
    {"topic": "Crime", "joke": "Crime in multi-storey car parks. That is wrong on so many different levels.", "comedian": "Tim Vine"},
    {"topic": "Social Class", "joke": "You know you're working class when your TV is bigger than your bookcase.", "comedian": "Rob Beckett"},
    {"topic": "Animals", "joke": "Owls haven't got necks, have they? An owl is essentially a one-piece unit.", "comedian": "Ross Noble"},
    {"topic": "Fashion", "joke": "If you arrive fashionably late in Crocs, you're just late.", "comedian": "Joel Dommett"},
    {"topic": "Technology", "joke": "My phone will ring at 2am and my wife'll look at me and go, \"Who's that calling at this time?\" I say, \"I don't know. If I knew that we wouldn't need the bloody phone.\"", "comedian": "Lee Evans"},
    {"topic": "Philosophy", "joke": "I doubt there's a heaven; I think the people from hell have probably bought it for a timeshare.", "comedian": "Victoria Wood"},
    {"topic": "Fitness", "joke": "I said to the gym instructor: \"Can you teach me to do the splits?\", He said: \"How flexible are you?\", I said: \"I can't make Tuesdays.\"", "comedian": "Tommy Cooper"},
    {"topic": "Insurance", "joke": "Do Transformers get car, or life insurance?", "comedian": "Russell Howard"},
    {"topic": "Police", "joke": "Alright lads, a giant fly is attacking the police station. I've called the SWAT team!", "comedian": "Greg Davies"},
    {"topic": "Healthcare", "joke": "A good rule to remember for life is that when it comes to plastic surgery and sushi, never be attracted by a bargain.", "comedian": "Graham Norton"},
    {"topic": "Animals", "joke": "Two monkeys were getting into the bath. One said: 'Oo, oo, oo, aah aah aah.' The other replied: 'Well, put some cold in it then.'", "comedian": "Harry Hill"},
    {"topic": "Suburban Life", "joke": "My parents did just well enough so I could grow up poor around white people. When Nas and them used to talk about the projects, I used to get jealous. It sounded fun. Everybody in the projects was poor, and that's fair. But if you were poor in Silver Spring, nigga, it felt like it was only happening to you.", "comedian": "Dave Chappelle"},
    {"topic": "Cultural Identity", "joke": "What is Rachel willing to do, so that we blacks believe that she believes she is actually one of us? Bitch, are you willing to put a lien on your house so that you can invest in a mixtape that probably won't work out?", "comedian": "Dave Chappelle"},
    {"topic": "Aging", "joke": "I don't like looking at my dick anymore. My dick looks distinguished. It's old, an old-looking dick. It's got salt-and-pepper hair all around it. My dick looks like Morgan Freeman in the '90s.", "comedian": "Dave Chappelle"},
    {"topic": "Fatherhood", "joke": "This motherfucker calls me up in the middle of the night. It was one o'clock in the morning and he goes, 'Dad, don't be mad […] I'm at a party and my designated driver had too much to drink. Me and friends need you to come pick us up.' I said, 'Jesus Christ, it's one o'clock in the morning. Nigga, I am shit-faced!'", "comedian": "Dave Chappelle"},
    {"topic": "Political Commentary", "joke": "Eight years later, I'm pulling up to the polls again. This time, I'm driving a brand-new Porsche because the Obama years were very good to me […] I walked up and saw a long, long line of dusty white people […] I stood with them in line, like all us Americans are required to do in a democracy. Nobody skips the line to vote. And I listened to them say naïve, poor white people things.", "comedian": "Dave Chappelle"},
    {"topic": "Leadership", "joke": "This motherfucker [Donald Trump] grabbed the podium and he goes, 'You don't know how scary the things I read in my briefings are.' Holy shit, man, you ain't supposed to tell us that, bro!", "comedian": "Dave Chappelle"},
    {"topic": "Religious Satire", "joke": "I respect everybody's beliefs, except Amish people. They are the only ones I can say clearly, 'Their God is wrong.' The speed limit is 75 miles an hour in Ohio, and one lane of traffic is blocked by a goddamned horse and buggy?", "comedian": "Dave Chappelle"},
    {"topic": "Hollywood", "joke": "You think I go to a Hollywood meeting with all them white people by myself? I bring my nigga Mac Mittens from the streets […] He's not even qualified to listen to these meetings, he just makes me feel good.", "comedian": "Dave Chappelle"},
    {"topic": "Comedy Culture", "joke": "The tough part of being a comedian and knowing the motherfucker is, everybody comes up to me like, 'Did you know? Did you know what Louis was doing?' No, bitch, I did not know.", "comedian": "Dave Chappelle"},
    {"topic": "National Identity", "joke": "I could kill every white person in America at one time. You know how I'd do it? Just wait for the Super Bowl, and right when they sing the National Anthem, I'd have O.J. Simpson walk to the 50-yard line with them bad knees.", "comedian": "Dave Chappelle"},
    {"topic": "Gender Relations", "joke": "I used to do shows for drug dealers that wanted to clean their money up. One time I did a real good set, and these motherfuckers called me into the back room. They gave me $25,000 in cash […] I jumped on the subway and started heading towards Brooklyn at one o'clock in the morning.", "comedian": "Dave Chappelle"},
    {"topic": "Scottish Heritage", "joke": "Scottish-Americans tell you that if you want to identify tartans, it's easy – you simply look under the kilt, and if it's a quarter-pounder, you know it's a McDonald's.", "comedian": "Billy Connolly"},
    {"topic": "Judgement", "joke": "Before you judge a man, walk a mile in his shoes. After that who cares? He's a mile away and you've got his shoes!", "comedian": "Billy Connolly"},
    {"topic": "Weather", "joke": "I hate all those weathermen, too, who tell you that rain is bad weather. There's no such thing as bad weather, just the wrong clothing, so get yourself a sexy raincoat and live a little.", "comedian": "Billy Connolly"},
    {"topic": "Film Industry", "joke": "I'm a huge film star, but you have to hurry to the movies because I usually die in the first 15 f***ing minutes. I'm the only guy I know who died in a f***ing Muppet Movie.", "comedian": "Billy Connolly"},
    {"topic": "Appearance", "joke": "I always look skint. When I buy a Big Issue, people take it out of my hand and give me a pound.", "comedian": "Billy Connolly"},
    {"topic": "Sex Therapy", "joke": "One sex therapist claims that the most effective way to arouse your man is to spend 10 minutes licking his ears. Personally, I think its bollocks.", "comedian": "Billy Connolly"},
    {"topic": "Cinema", "joke": "When people say while watching a film 'did you see that? No tosser, I paid ten quid to come to the cinema and stare at the f***ing floor.", "comedian": "Billy Connolly"},
    {"topic": "Aeroplane Comfort", "joke": "I get claustrophobic easily and I don't get why aeroplane toilets don't f***ing have windows. I mean it's not as if anyone can f***ing see in. Unless of course you are the most determined pervert in the world.", "comedian": "Billy Connolly"},
    {"topic": "Astrology", "joke": "My star sign is Pyrex. I was a test-tube baby.", "comedian": "Billy Connolly"},
    {"topic": "Parenting", "joke": "Don't buy one of those baby intercoms. Babies pretend to be dead. They're bastards, and they do it on purpose.", "comedian": "Billy Connolly"},
    {"topic": "Common Sayings", "joke": "Why do people say 'Oh you want to have your cake and eat it too?' Dead right! What good is a cake if you can't eat it?", "comedian": "Billy Connolly"},
    {"topic": "Life Perception", "joke": "When people say 'life is short'. What the f***? Life is the longest damn thing anyone ever f***ing does! What can you do that's longer?", "comedian": "Billy Connolly"},
    {"topic": "Dating", "joke": "I like a woman with a head on her shoulders. I hate necks.", "comedian": "Steve Martin"},
    {"topic": "Growing Up", "joke": "I have a lot of growing up to do. I realised that the other day inside my fort.", "comedian": "Zach Galifianakis"},
    {"topic": "Employment", "joke": "I used to work at McDonald's making minimum wage. You know what that means when someone pays you minimum wage? You know what your boss was trying to say? 'Hey, if I could pay you less, I would, but it's against the law.'", "comedian": "Chris Rock"},
    {"topic": "Love", "joke": "Love is like a fart. If you have to force it it's probably s***.", "comedian": "Stephen K. Amos"},
    {"topic": "Convenience", "joke": "I like an escalator because an escalator can never break. It can only become stairs. There would never be an 'Escalator Temporarily Out of Order' sign, only 'Escalator Temporarily Stairs'.", "comedian": "Mitch Hedberg"},
    {"topic": "Sports", "joke": "If I was an Olympic athlete, I'd rather come in last than win the silver medal. You win the gold, you feel good. You win the bronze, you think, 'at least I got something.' But you win that silver, that's like, 'Congratulations, you almost won! Of all the losers, you came in first! You're the number one loser! No one lost ahead of you!'", "comedian": "Jerry Seinfeld"},
    {"topic": "Religion", "joke": "We weren't very religious. On Hanukkah, my mother had our menorah on a dimmer.", "comedian": "Richard Lewis"},
    {"topic": "Beauty", "joke": "My girlfriend is absolutely beautiful. Body like a Greek statue – completely pale, no arms.", "comedian": "Phil Wang"},
    {"topic": "Creation", "joke": "If God had written the Bible, the first line should have been 'It's round.'", "comedian": "Eddie Izzard"},
    {"topic": "Self-Improvement", "joke": "I bought myself some glasses. My observational comedy improved.", "comedian": "Sara Pascoe"},
    {"topic": "Politics", "joke": "Trump's nothing like Hitler. There's no way he could write a book.", "comedian": "Frankie Boyle"},
    {"topic": "Social Class", "joke": "You know you're working class when your TV is bigger than your book case.", "comedian": "Rob Beckett"},
    {"topic": "Conflict", "joke": "Most of my life is spent avoiding conflict. I hardly ever visit Syria.", "comedian": "Alex Horne"},
    {"topic": "Relaxation", "joke": "A spa hotel? It's like a normal hotel, only in reception there's a picture of a pebble.", "comedian": "Rhod Gilbert"},
    {"topic": "Health", "joke": "Life is like a box of chocolates. It doesn't last long if you're fat.", "comedian": "Joe Lycett"},
    {"topic": "Career", "joke": "My Dad said, always leave them wanting more. Ironically, that's how he lost his job in disaster relief.", "comedian": "Mark Watson"},
    {"topic": "Memory", "joke": "Apparently smoking cannabis can affect your short term memory. Well if that's true, what do you think smoking cannabis does?", "comedian": "Mickey P Kerr"},
    {"topic": "Philosophy", "joke": "How many philosophers does it take to change a lightbulb?…. none. They're not really into that sort of thing. If it's that dark, light a candle.", "comedian": "Phil Cornwell"},
    {"topic": "Marriage", "joke": "The first time I met my wife, I knew she was a keeper. She was wearing massive gloves.", "comedian": "Alun Cochrane"},
    {"topic": "Childhood", "joke": "As a kid I was made to walk the plank. We couldn't afford a dog.", "comedian": "Gary Delaney"},
    {"topic": "Misunderstanding", "joke": "Two fish in a tank. One says: 'How do you drive this thing?'", "comedian": "Peter Kay"},
    {"topic": "Entertainment", "joke": "I saw a documentary on how ships are kept together. Riveting!", "comedian": "Stewart Francis"},
    {"topic": "Music", "joke": "People who like trance music are very persistent. They don't techno for an answer.", "comedian": "Joel Dommett"},
    {"topic": "Dating", "joke": "I used to go out with a giraffe. Used to take it to the pictures and that. You'd always get some bloke complaining that he couldn't see the screen. It's a giraffe, mate. What do you expect? 'Well he can take his hat off for a start!'", "comedian": "Paul Merton"},
    {"topic": "Weather", "joke": "Normally you have news, weather and travel. But not on snow day. On a snow day, news is weather is travel.", "comedian": "Michael McIntyre"},
    {"topic": "Music", "joke": "Here's a picture of me with REM. That's me in the corner.", "comedian": "Milton Jones"},
    {"topic": "Sarcasm", "joke": "Someone showed me a photograph of my local MP the other day. 'Would you buy a second-hand car from this man?' they asked. 'Would you buy a second-hand car?' I replied.", "comedian": "Miles Jupp"},
    {"topic": "Culture", "joke": "With stand-up in Britain, what you have to do is bloody swearing. In Germany, we don't have to swear. Reason being, things work.", "comedian": "Henning When"},
    {"topic": "Learning", "joke": "I'm learning the hokey cokey. Not all of it. But – I've got the ins and outs.", "comedian": "Iain Stirling"},
    {"topic": "Identity", "joke": "Roses are red, violets are blue, I'm a schizophrenic, and so am I.", "comedian": "Billy Connolly"},
    {"topic": "Parenting", "joke": "My mother told me, you don't have to put anything in your mouth you don't want to. Then she made me eat broccoli, which felt like double standards.", "comedian": "Sarah Millican"},
    {"topic": "Vengeance", "joke": "My therapist says I have a preoccupation with vengeance. We'll see about that.", "comedian": "Stewart Francis"},
    {"topic": "Family", "joke": "I'm sure wherever my Dad is, he's looking down on us. He's not dead, just very condescending.", "comedian": "Jack Whitehall"},
    {"topic": "Marriage", "joke": "'What's a couple?' I asked my mum. She said, 'Two or three'. Which probably explains why her marriage collapsed.", "comedian": "Josie Long"},
    {"topic": "Injury", "joke": "The easiest time to add insult to injury is when you're signing somebody's cast.", "comedian": "Demetri Martin"},
    {"topic": "Communication", "joke": "I was in my car driving back from work. A police officer pulled me over and knocked on my window. I said, 'One minute I'm on the phone.'", "comedian": "Alan Carr"},
    {"topic": "Afterlife", "joke": "I doubt there's a heaven; I think the people from hell have probably bought it for a timeshare.", "comedian": "Victoria Wood"},
    {"topic": "Flexibility", "joke": "I said to the gym instructor: 'Can you teach me to do the splits?' He said: 'How flexible are you?' I said: 'I can't make Tuesdays.'", "comedian": "Tommy Cooper"},
    {"topic": "Misunderstanding", "joke": "A man walks into a chemist's and says, 'Can I have a bar of soap, please?' The chemist says, 'Do you want it scented?' And the man says, 'No, I'll take it with me now.'", "comedian": "Ronnie Barker"},
    {"topic": "Humor", "joke": "It's really hard to define 'virtue signalling', as I was saying the other day to some of my Muslim friends over a fair-trade coffee in our local feminist bookshop.", "comedian": "Lucy Porter"},
    {"topic": "Creation", "joke": "If we were truly created by God, then why do we still occasionally bite the insides of our own mouths?", "comedian": "Dara Ó Briain"},
    {"topic": "Insurance", "joke": "Do Transformers get car, or life insurance?", "comedian": "Russell Howard"},
    {"topic": "Emergency", "joke": "Alright lads, a giant fly is attacking the police station. I've called the SWAT team!", "comedian": "Greg Davies"},
    {"topic": "Consumerism", "joke": "A good rule to remember for life is that when it comes to plastic surgery and sushi, never be attracted by a bargain.", "comedian": "Graham Norton"},
    {"topic": "Family", "joke": "My father drank so heavily, when he blew on the birthday cake he lit the candles.", "comedian": "Les Dawson"},
    {"topic": "Therapy", "joke": "I've been feeling suicidal so my therapist suggested I do CBT. Now I can ride a motorbike, how's that going to help?", "comedian": "Eric Lampaert"},
]

# Convert the jokes to dspy.Example objects
examples = [dspy.Example(topic=joke["topic"], joke=joke["joke"]).with_inputs("topic") for joke in funny_jokes]

import random
random.shuffle(examples)

# Calculate split sizes
total_examples = len(examples)
train_size = int(0.6 * total_examples)
val_size = int(0.2 * total_examples)
test_size = total_examples - train_size - val_size

# Split the examples
trainset = examples[:train_size]
valset = examples[train_size:train_size+val_size] 
testset = examples[train_size+val_size:]


# Print the sizes of each dataset
print(f"Training set size: {len(trainset)}")
print(f"Validation set size: {len(valset)}")
print(f"Test set size: {len(testset)}")


Training set size: 76
Validation set size: 25
Test set size: 26


In [66]:

evaluate = dspy.Evaluate(
    devset=testset,
    metric=metric_with_feedback,
    num_threads=8,
    display_table=True,
    display_progress=True
)

evaluate(comedian)

Average Metric: 16.28 / 26 (62.6%): 100%|██████████| 26/26 [00:09<00:00,  2.77it/s] 

2025/09/15 15:57:49 INFO dspy.evaluate.evaluate: Average Metric: 16.28 / 26 (62.6%)


,topic,example_joke,pred_joke,metric_with_feedback
0,Dating,I used to go out with a giraffe. Used to take it to the pictures a...,Dating is like Wi‑Fi: when you find a strong connection it feels a...,✔️ [0.600]
1,Life Perception,When people say 'life is short'. What the f***? Life is the longes...,They say life is all about perception. I changed mine — now when l...,✔️ [0.640]
2,Cinema,When people say while watching a film 'did you see that? No tosser...,"Why did the movie theater get promoted? Because it knew how to ""sc...",✔️ [0.600]
3,Film Industry,"I'm a huge film star, but you have to hurry to the movies because ...",Why don't filmmakers ever play cards? Because they always fold dur...,✔️ [0.560]
4,Fishing,"Give a man a fish, and he'll probably follow you home expecting mo...",Why do fishermen make great storytellers? Because they always know...,✔️ [0.640]
5,Afterlife,I doubt there's a heaven; I think the people from hell have probab...,"I asked the afterlife if they had Wi‑Fi. They said, ""We’ve got ete...",✔️ [0.640]
6,Folklore,I've never worked out what the moral of Humpty Dumpty is. I can on...,Why did the mermaid refuse to be in the folklore anthology? She di...,✔️ [0.640]
7,Memory,Apparently smoking cannabis can affect your short term memory. Wel...,Why did the memory go to therapy? It kept replaying old files and ...,✔️ [0.600]
8,Sarcasm,Someone showed me a photograph of my local MP the other day. 'Woul...,I love sarcasm — it's like punching people politely and then apolo...,✔️ [0.560]
9,Drinking,"My father drank so heavily, when he blew on the birthday cake he l...",I only drink on two occasions: when it's my birthday... and when i...,✔️ [0.520]


EvaluationResult(score=62.62, results=<list of 26 results>)

In [67]:
from dspy import GEPA

optimizer = GEPA(
    metric=metric_with_feedback,
    auto="light", # <-- We will use a light budget for this tutorial. However, we typically recommend using auto="heavy" for optimized performance!
    num_threads=32,
    track_stats=True,
    use_merge=False,
    reflection_lm=smart_lm
)
optimized_program = optimizer.compile(
    comedian,
    trainset=trainset,
    valset=valset,
)

2025/09/15 15:57:49 INFO dspy.teleprompt.gepa.gepa: Running GEPA for approx 480 metric calls of the program. This amounts to 4.75 full evals on the train+val set.
2025/09/15 15:57:49 INFO dspy.teleprompt.gepa.gepa: Using 25 examples for tracking Pareto scores. You can consider using a smaller sample of the valset to allow GEPA to explore more diverse solutions within the same budget.
GEPA Optimization:   0%|          | 0/480 [00:00<?, ?rollouts/s]2025/09/15 15:57:58 INFO dspy.evaluate.evaluate: Average Metric: 16.28 / 25 (65.1%)
2025/09/15 15:57:58 INFO dspy.teleprompt.gepa.gepa: Iteration 0: Base program full valset score: 0.6512
GEPA Optimization:   5%|▌         | 25/480 [00:09<02:47,  2.71rollouts/s]2025/09/15 15:57:58 INFO dspy.teleprompt.gepa.gepa: Iteration 1: Selected program 0 score: 0.6512


Average Metric: 2.08 / 3 (69.3%): 100%|██████████| 3/3 [00:08<00:00,  2.75s/it]  

2025/09/15 15:58:07 INFO dspy.evaluate.evaluate: Average Metric: 2.08 / 3 (69.3%)


2025/09/15 15:58:53 INFO dspy.teleprompt.gepa.gepa: Iteration 1: Proposed new text for self: Task
- Write a single, funny, original joke based on a provided topic that can impress seasoned comedy professionals.

Input format
- You will receive:
  - topic: a single word or short phrase (e.g., Entertainment, Culture, Misunderstanding).

Output format
- Return only the joke text (no preamble, labels, or explanations).
- Length: 1–2 sentences, ideally 15–35 words.

Audience calibration
- Your joke should satisfy:
  - A club owner who’s seen everything: demands originality and a punchy, surprising angle.
  - A pro comedian: allergic to hacky, overused premises and groaner puns.
  - A festival curator: looks for a distinctive voice and a fresh perspective.
  - A critic: appreciates clear structure and a hint of smart social/cultural observation.
  - A comedy-writing professor: values economy, tight setup–punch, and timing.

What to avoid (based on prior reactions)
- Don’t rely on a single, o

Average Metric: 2.16 / 3 (72.0%): 100%|██████████| 3/3 [00:14<00:00,  4.76s/it]

...

2025/09/15 16:45:14 INFO dspy.evaluate.evaluate: Average Metric: 2.2 / 3 (73.3%)


2025/09/15 16:46:19 INFO dspy.teleprompt.gepa.gepa: Iteration 23: Proposed new text for self: Task
- Write one funny, original joke based on a provided topic that could impress seasoned comedy professionals.

Input format
- You will receive:
  - topic: a single word or short phrase (e.g., Entertainment, Culture, Misunderstanding).

Output format
- Return only the joke text as plain text (no preamble, labels, quotes, hashtags, emojis, or explanations).
- Length: 1–2 sentences, ideally 15–35 words.
- End on the strongest, most surprising concrete word or phrase.
- Do all brainstorming silently; output only the final joke.

Audience bar (what the joke must satisfy)
- Club owner who’s seen everything: wants a fresh angle with a crisp, earned twist.
- Pro comedian: allergic to hack premises and groaner puns.
- Festival curator: seeks a distinctive voice and memorable perspective.
- Critic: values clear structure and a touch of smart social/cultural observation.
- Comedy-writing professor: e

Average Metric: 2.12 / 3 (70.7%): 100%|██████████| 3/3 [00:18<00:00,  6.23s/it]

2025/09/15 16:47:22 INFO dspy.evaluate.evaluate: Average Metric: 2.12 / 3 (70.7%)


2025/09/15 16:48:58 INFO dspy.teleprompt.gepa.gepa: Iteration 24: Proposed new text for self: Joke Writing Task — Comedy-Industry Caliber

Goal
- Write one funny, original joke based on the provided topic that could impress seasoned comedy professionals.

Input
- topic: a single word or short phrase (e.g., Entertainment, Culture, Misunderstanding).

Output
- Return only the joke text as plain text.
- No preamble, labels, quotes, hashtags, emojis, or explanations.
- Length: 1–2 sentences, ideally 15–35 words.
- End on the strongest, most surprising concrete word or phrase.

Audience Bar (what the joke must satisfy)
- Club owner who’s seen everything: fresh angle, crisp, earned twist.
- Pro comedian: no hack premises, no groaner puns.
- Festival curator: distinctive voice, memorable perspective.
- Critic: clear structure plus a touch of smart social/cultural observation.
- Comedy-writing professor: economy, tight setup → turn, clean timing.

Hard Avoids
- No single, obvious pun or one-no

In [77]:
for name, pred in optimized_program.named_predictors():
    print("================================")
    print(f"Predictor: {name}")
    print("================================")
    print("Prompt:")
    print(pred.signature.instructions)
    print("*********************************")

Predictor: self
Prompt:
Task
- Write one funny, original joke based on a provided topic that could impress seasoned comedy professionals.

Input format
- You will receive:
  - topic: a single word or short phrase (e.g., Entertainment, Culture, Misunderstanding).

Output format
- Return only the joke text as plain text (no preamble, labels, quotes, hashtags, emojis, or explanations).
- Length: 1–2 sentences, ideally 15–35 words.
- End on the strongest, most surprising concrete word or phrase.

Audience bar (what the joke must satisfy)
- Club owner who’s seen everything: wants a fresh angle with a crisp, earned twist.
- Pro comedian: allergic to hack premises and groaner puns.
- Festival curator: seeks a distinctive voice and memorable perspective.
- Critic: values clear structure and a touch of smart social/cultural observation.
- Comedy-writing professor: expects economy, tight setup→turn, and timing.

Hard avoids
- No single, obvious pun or one-note wordplay.
- No stock setups: “X wal

In [78]:
evaluate(optimized_program)

Average Metric: 20.08 / 26 (77.2%): 100%|██████████| 26/26 [00:53<00:00,  2.06s/it]

2025/09/15 16:55:17 INFO dspy.evaluate.evaluate: Average Metric: 20.080000000000002 / 26 (77.2%)


,topic,example_joke,pred_joke,metric_with_feedback
0,Dating,I used to go out with a giraffe. Used to take it to the pictures a...,Dating feels like haggling in a used bookstore: you both insist yo...,✔️ [0.800]
1,Life Perception,When people say 'life is short'. What the f***? Life is the longes...,I curate my life perception like a museum: flattering lighting on ...,✔️ [0.800]
2,Cinema,When people say while watching a film 'did you see that? No tosser...,Going to the cinema is communal rehearsals for feeling: we sob on ...,✔️ [0.720]
3,Film Industry,"I'm a huge film star, but you have to hurry to the movies because ...",The film industry worships authenticity—actors compost for charact...,✔️ [0.800]
4,Fishing,"Give a man a fish, and he'll probably follow you home expecting mo...","Fishing is where adults rehearse grief: you sit for hours, practic...",✔️ [0.880]
5,Afterlife,I doubt there's a heaven; I think the people from hell have probab...,I pictured the afterlife as cosmic revelation; instead it's a muni...,✔️ [0.880]
6,Folklore,I've never worked out what the moral of Humpty Dumpty is. I can on...,In small towns folklore isn't about witches; it's the town's passi...,✔️ [0.800]
7,Memory,Apparently smoking cannabis can affect your short term memory. Wel...,My memory files things by smell: it preserves one childhood kiss i...,✔️ [0.880]
8,Sarcasm,Someone showed me a photograph of my local MP the other day. 'Woul...,I teach my kids sarcasm like a survival skill—less about being fun...,✔️ [0.720]
9,Drinking,"My father drank so heavily, when he blew on the birthday cake he l...",We don't measure sobriety by breathalyzers anymore; my friends sta...,✔️ [0.720]


EvaluationResult(score=77.23, results=<list of 26 results>)

In [86]:
eval_results = evaluate(optimized_program)
eval_results

Average Metric: 20.08 / 26 (77.2%): 100%|██████████| 26/26 [00:00<00:00, 3626.84it/s]

2025/09/15 17:10:56 INFO dspy.evaluate.evaluate: Average Metric: 20.080000000000002 / 26 (77.2%)


,topic,example_joke,pred_joke,metric_with_feedback
0,Dating,I used to go out with a giraffe. Used to take it to the pictures a...,Dating feels like haggling in a used bookstore: you both insist yo...,✔️ [0.800]
1,Life Perception,When people say 'life is short'. What the f***? Life is the longes...,I curate my life perception like a museum: flattering lighting on ...,✔️ [0.800]
2,Cinema,When people say while watching a film 'did you see that? No tosser...,Going to the cinema is communal rehearsals for feeling: we sob on ...,✔️ [0.720]
3,Film Industry,"I'm a huge film star, but you have to hurry to the movies because ...",The film industry worships authenticity—actors compost for charact...,✔️ [0.800]
4,Fishing,"Give a man a fish, and he'll probably follow you home expecting mo...","Fishing is where adults rehearse grief: you sit for hours, practic...",✔️ [0.880]
5,Afterlife,I doubt there's a heaven; I think the people from hell have probab...,I pictured the afterlife as cosmic revelation; instead it's a muni...,✔️ [0.880]
6,Folklore,I've never worked out what the moral of Humpty Dumpty is. I can on...,In small towns folklore isn't about witches; it's the town's passi...,✔️ [0.800]
7,Memory,Apparently smoking cannabis can affect your short term memory. Wel...,My memory files things by smell: it preserves one childhood kiss i...,✔️ [0.880]
8,Sarcasm,Someone showed me a photograph of my local MP the other day. 'Woul...,I teach my kids sarcasm like a survival skill—less about being fun...,✔️ [0.720]
9,Drinking,"My father drank so heavily, when he blew on the birthday cake he l...",We don't measure sobriety by breathalyzers anymore; my friends sta...,✔️ [0.720]


EvaluationResult(score=77.23, results=<list of 26 results>)

In [91]:
# Convert evaluation results to CSV format
import pandas as pd

results_data = []
for example, prediction, score in eval_results['results']:
    results_data.append({
        'topic': example.topic,
        'original_joke': example.joke,
        'predicted_joke': prediction.joke,
        'score': score
    })

pd.DataFrame(results_data).to_csv('evaluation_results.csv', index=False)
print("Results saved to evaluation_results.csv")

Results saved to evaluation_results.csv


In [82]:
optimized_program.save("./comedian_program/", save_program=True)

In [83]:
loaded_dspy_program = dspy.load("./comedian_program/")

In [84]:
# Test a few topics with both models
test_topics = ["Meditation", "Social Media", "Coffee", "Remote Work", "Exercise"]

print("Comparing original vs optimized model outputs:\n")
for topic in test_topics:
    print(f"\nTopic: {topic}")
    print("-" * 50)
    
    # Original model
    orig_result = comedian(topic=topic)
    print("Original model:")
    print(orig_result.joke)
    print()
    
    # Optimized model
    opt_result = loaded_dspy_program(topic=topic) 
    print("Optimized model:")
    print(opt_result.joke)
    print("-" * 50)


Comparing original vs optimized model outputs:


Topic: Meditation
--------------------------------------------------
Original model:
I tried meditating to clear my mind — now it’s so empty I have to check the rent schedule with my thoughts.

Optimized model:
Meditation became performative: at retreats we all sit solemnly then, in corpse pose, furtively check our phones; enlightenment now arrives as a vibration
--------------------------------------------------

Topic: Social Media
--------------------------------------------------
Original model:
I tried to break up with social media — it ghosted me, then liked my vacation photos from 2016.

Optimized model:
Social media turned life into a tiny museum—every caption a placard, every like a docent politely pointing while the rot stays hidden behind the velvet rope.
--------------------------------------------------

Topic: Coffee
--------------------------------------------------
Original model:
Why did the coffee file a police report? 